In [1]:
import requests
import json
import pandas as pd
import time
import convex as cx
import tmqa35 as tmqa
import importlib
from datetime import datetime

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
#importlib.reload(tmqa)

In [3]:
#f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
#out_test = f_test.readlines()

In [4]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [5]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [6]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [7]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [8]:
def ask_tmqa(question, verbose=False, timer=False, show_graph=False, cores=tmqa.mp.cpu_count(), banning_str=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose, timer=timer, show_graph=show_graph, cores=cores, banning_str=banning_str)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    if tmqa_result == (False,False):
        return False
    #if tmqa_result[]
    #if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
    #    return False
    return tmqa_result[0][0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#answer = ask_tmqa("Who is the wife of Barack Obama")
#print(answer)

In [9]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'tmqa']
#df = pd.DataFrame(columns=HEADERS)

#start_time = time.time()
#out_test_len = len(out_test)
##print(out_test_len)
#for i, line in enumerate(out_test):
#    arguments = line.split("\t")
#    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
#    if i < len(df):
#        continue
#    result_qanswer = ask_qanswer(arguments[3][:-1])
#    result_platypus = ask_platypus(arguments[3][:-1])
#    result_convex = ask_convex(arguments[3][:-1])
#    #result_tmqa = ask_tmqa(arguments[3][:-1])[1]
#    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
#    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
#    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
#    #print("\n",i, "-> TMqa", result_tmqa) if result_tmqa == arguments[2] else False
#    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'tmqa1':result_tmqa}, ignore_index=True)
#
#end_time = time.time()
#print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
#print(df)

In [10]:
#len(df)

In [ ]:
#df.tail(5)

In [ ]:
#SAVING
#pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(len(df_loaded)))

In [ ]:
#LOADING
#df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-9961.pickle.bz2")

#/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-5.pickle.bz2


In [ ]:
#df = df_loaded

In [ ]:
#df_loaded

In [ ]:
#df_loaded = df_loaded.replace("", False)

In [ ]:
#df_loaded['qanswer'][34] = False

In [ ]:
#del df_loaded['tm2']

In [ ]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [ ]:
#df_loaded['tm1_top4'] = "False"

In [ ]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [ ]:
#df_loaded

In [11]:
#LOADING QAGraph

df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-6455.pickle.bz2")



In [12]:
print(df_loaded.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
banning_str = False#[["ř","r"],["ø","o"]]

df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 6457:
        print("It is ", datetime.now())
        
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa = ask_tmqa(question, verbose=True, timer=True, show_graph=False, banning_str=banning_str) 
        if result_tmqa:
            df_loaded['tm1'][i] = result_tmqa[0]
            if source in result_tmqa[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa:",str(source))
        
        pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(i))
        
        print(df_loaded.loc[i,:])
        print("\n")
        
        #break

It is  2020-03-09 19:43:01.136405
6457/9961 what is a game with single_player as its gameplay mode -> source: Q7602433
User input: what is a game with single_player as its gameplay mode
--> Auto correcting question in progress...
-> Auto corrected q_nlp: What is a game with single player as its gameplay mode 
-> q_themes: ([(a game, ['Q4646897']), (game, ['Q11410', 'Q15079592'])], [its gameplay mode, single_player, Single_player, Its Gameplay Mode, gameplay mode, its Gameplay Mode])
-> q_themes_enhanced: [('gameplay', ['P404']), ('mode', ['Q188224']), ('Gameplay', ['Q2720327']), ('Mode', ['Q11883080'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: single
behold: get_most_similar started with: player
-> q_predicates: [(be, ['P31']), (game, ['P1350']), (single, ['P555']), (player, ['P1873', 'P1872']), (gameplay, ['P404']), (mode, ['P404'])]
-> q_predicates 	Running time is 11.23s
--> Potential meaningful keyw

In [ ]:
df_loaded.head(4486)

In [ ]:
df_copy = df_loaded.copy()
#df = df.replace("", False)

In [ ]:
max_row = 6454
df_len = len(df_copy)
df_qanswer_max = df_copy[(df_copy.index<=max_row) & (df_copy.qanswer == df_copy.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df_copy[(df_copy.index<=max_row) & (df_copy.platypus == df_copy.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df_copy[(df_copy.index<=max_row) & (df_copy.convex == df_copy.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df_copy[(df_copy.index<=max_row) & (df_copy.tm1 == df_copy.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

In [ ]:
max_row = 6455

df_qanswer_max = df_copy[(df_copy.index<=max_row) & (df_copy.qanswer == df_copy.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df_copy[(df_copy.index<=max_row) & (df_copy.platypus == df_copy.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df_copy[(df_copy.index<=max_row) & (df_copy.convex == df_copy.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df_copy[(df_copy.index<=max_row) & (df_copy.tm1 == df_copy.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("PRECISION")
print("qanswer:", df_qanswer_max_len,df_qanswer_max_len)
print("platypus:", df_platypus_max_len, df_platypus_max_len)
print("convex:", df_convex_max_len, df_convex_max_len)
print("tm1:", df_tm1_max_len, df_tm1_max_len)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

In [ ]:
max_row = 6455

df_qanswer_recall = (max_row-len(df_copy[(df_copy.index<=max_row) & (df_copy.qanswer == False)]))/max_row
df_platypus_recall = (max_row-len(df_copy[(df_copy.index<=max_row) & (df_copy.platypus == False)]))/max_row
df_convex_recall = (max_row-len(df_copy[(df_copy.index<=max_row) & (df_copy.convex == False)]))/max_row
df_tm1_recall = (max_row-len(df_copy[(df_copy.index<=max_row) & (df_copy.tm1 == False)]))/max_row

df_qanswer_precision = len(df_copy[(df_copy.index<=max_row) & (df_copy.qanswer == df_copy.source)])/max_row
df_platypus_precision = len(df_copy[(df_copy.index<=max_row) & (df_copy.platypus == df_copy.source)])/max_row
df_convex_precision = len(df_copy[(df_copy.index<=max_row) & (df_copy.convex == df_copy.source)])/max_row
df_tm1_precision = len(df_copy[(df_copy.index<=max_row) & (df_copy.tm1 == df_copy.source)])/max_row



print("RECALL")
print("qanswer:", df_qanswer_recall)
print("platypus:", df_platypus_recall)
print("convex:", df_convex_recall)
print("tm1:", df_tm1_recall)
                            
                            
print("PRECISION")
print("qanswer:", df_qanswer_precision)
print("platypus:", df_platypus_precision)
print("convex:", df_convex_precision)
print("tm1:", df_tm1_precision)

print("F1")
print("qanswer:", 2*((df_qanswer_precision*df_qanswer_recall)/(df_qanswer_precision+df_qanswer_recall)))
print("platypus:", 2*((df_platypus_precision*df_platypus_recall)/(df_platypus_precision+df_platypus_recall)))
print("convex:", 2*((df_convex_precision*df_convex_recall)/(df_convex_precision+df_convex_recall)))
print("tm1:", 2*((df_tm1_precision*df_tm1_recall)/(df_tm1_precision+df_tm1_recall)))


#df[ & (df.qanswer == df.source)]

In [ ]:
print("qanswer:",len(df_copy[df_copy.qanswer == df_copy.source]),len(df_copy[df_copy.qanswer == df_copy.source])/len(df_copy))
print("platypus:",len(df_copy[df_copy.platypus == df_copy.source]),len(df_copy[df_copy.platypus == df_copy.source])/len(df_copy))
print("convex:",len(df_copy[df_copy.convex == df_copy.source]),len(df_copy[df_copy.convex == df_copy.source])/len(df_copy))
print("tm1:",len(df_copy[df_copy.tm1 == df_copy.source]),len(df_copy[df_copy.tm1 == df_copy.source])/len(df_copy))
print("tm1_top2:",len(df_copy[df_copy.tm1_top2 == True]),len(df_copy[df_copy.tm1_top2 == True])/len(df_copy))
print("tm1_top3:",len(df_copy[df_copy.tm1_top3 == True]),len(df_copy[df_copy.tm1_top3 == True])/len(df_copy))
print("tm1_top4:",len(df_copy[df_copy.tm1_top4 == True]),len(df_copy[df_copy.tm1_top4 == True])/len(df_copy))
print("tm1_top5:",len(df_copy[df_copy.tm1_top5 == True]),len(df_copy[df_copy.tm1_top5 == True])/len(df_copy))
print("tm1_topall:",len(df_copy[df_copy.tm1_topall == True]),len(df_copy[df_copy.tm1_topall == True])/len(df_copy))



In [ ]:
#df.tm1_top2 = df.tm1_top3